# Create a projection from the IOC data

There a 3 components to a projection with the poses:
1. Features from the feature table (Feature Table)
2. A Projection (Projection table)
3. Mappings of the coordinates to the projection and features (MapProjectionFeature Table)

As the keypoints and angles are already provided in the associated feature data, the atlas is not needed to create the projection.

In [12]:
from emv.db.dao import DataAccessObject
from sqlalchemy.sql import text
import pandas as pd
import ast
import numpy as np
from emv.api.models import Projection, MapProjectionFeatureCreate
from emv.db.queries import create_projection, create_map_projection_feature
import umap

In [6]:
# Retrieve the transformed features for the poses, feature_type = 'pose_image' is a smaller dataset than just 'pose'
# In comparison to the full data, it has normalized keypoints and embeddings
query = text("SELECT * FROM feature WHERE feature_type = 'pose_image'")
df = pd.DataFrame(DataAccessObject().fetch_all(query))
df['embedding_33'] = df['embedding_33'].apply(lambda x: ast.literal_eval(x))

In [ ]:
# default values that assume an atlas, not necessary, 
# but it's good to give the correct values when creating the projection

total_tiles = len(df) # either all features or a subset of features
atlas_width = 4096
max_tile_size = 512
max_tiles_per_atlas = (atlas_width // max_tile_size) ** 2
atlas_count = int(total_tiles / max_tiles_per_atlas) + 1

In [ ]:
# Create the projection, replace the names with the desired ones, library_id = 2 is for the IOC
projection = Projection(
    projection_name="IOC Poses",
    version="0.0.1",
    library_id=2,
    model_name="openpifpaf_fast",
    model_params={},
    data={},
    dimension=3,
    atlas_folder_path="",
    atlas_width=atlas_width,
    tile_size=max_tile_size,
    atlas_count=atlas_count,
    total_tiles=total_tiles,
    tiles_per_atlas=max_tiles_per_atlas,
)

projection_id = create_projection(projection)['projection_id']

In [15]:
# Replace with a different way to come up with coordinates, coordinates can be 2D or 3D
reducer = umap.UMAP(n_components=3)
data = np.array(df['embedding_33'].tolist())
embedding = reducer.fit_transform(data)

In [ ]:
# Create an entry in the map_projection_feature table for each feature, links features, media and coordinates
for i, row in df.iterrows():
    create_map_projection_feature(MapProjectionFeatureCreate(
        projection_id=projection_id,
        media_id=row.media_id,
        atlas_order=-1,
        index_in_atlas=-1,
        coordinates=[embedding[i, 0], embedding[i, 1], embedding[i, 2]],
        feature_id=row.feature_id
    ))